# Basic performance considerations

## Type stability

Julia matches the performance of C/Fortran not because of better hardware or better compilers than e.g., Python uses, but because of design. Julia is interactive like Python, but it is not an interpreted language, it is a **compiled** one. This means that **every function call in Julia first gets compiled, based on the exact input types**. Then it is executed.

*(this compilation only happens once for each unique combination of input types)*

When the compiler compiles a function, these types of every variable can be tracked throughout the function and all datastructures are mapped uniquely and deterministically all the way from input to output. This allows the compiler to make all the optimizations that e.g. the compilation of a C language code would do. And this (in a nutshell) is what results in the same performance as C/Fortran.

This tracking of types mentioned above only works if **the type of every variable remains the same type throughout the function's operations**. Notice the distinction: the _type_ (i.e. all floating point variables remain floats) is constant, but of course the _value_ could change (i.e. going from `515134.515` to `123415.242` is fine).

What if this doesn't happen? Then we have the case of **Type instability**, which is what makes beginner's code slow in 99% of the cases. 

Let's look at the following illustrative scenario:

In [ ]:
Pkg.activate(@__DIR__)
using BenchmarkTools

  Activating project at `c:\Users\datse\OneDrive - University of Exeter\Teaching\Zero2Hero-JuliaWorkshop`


In [ ]:
@noinline function mymax(init, x)
    s = init
    for xi in x
        s = s < xi ? xi : s
    end
    return s
end

# benchmark it
using BenchmarkTools: @btime
r = rand(Int, 1000);
@btime mymax(-Inf32, $r);

  5.067 μs (0 allocations: 0 bytes)


In [ ]:
@btime mymax(-Inf, $r);

  728.788 ns (0 allocations: 0 bytes)


In [ ]:
@btime mymax(typemin(Int), $r);

  72.895 ns (0 allocations: 0 bytes)


`mymax` is a **type-unstable function**, which means it has suboptimal performance. 

you can see this by using the `@code_warntype` macro

In [ ]:
@code_warntype mymax(-Inf, r)

MethodInstance for mymax(::Float64, ::Vector{Int64})
  from mymax(init, x) @ Main c:\Users\datse\OneDrive - University of Exeter\Teaching\Zero2Hero-JuliaWorkshop\1-JuliaIntro.ipynb:1
Arguments
  

#self#::Core.Const(mymax)
  init::Float64
  x::Vector{Int64}
Locals
  @_4::Union{Nothing, Tuple{Int64, Int64}}
  s::Union{Float64, Int64}
  xi::Int64
  @_7::Union{Float64, Int64}
Body::Union{Float64, Int64}


1 ─       

$(Expr(:meta, :noinline))
│         

(s = init)
│   %3  = x::Vector{Int64}
│         (@_4 = Base.iterate(%3))
│   %5  = (@_4 === nothing)::Bool
│   %6  = Base.not_int(%5)::Bool
└──       goto #7 if not %6
2 ┄ %8  = @_4::Tuple{Int64, Int64}
│         (xi = Core.getfield(%8, 1))
│   %10 = Core.getfield(%8, 2)::Int64
│   %11 = (s < xi)::Bool
└──       goto #4 if not %11
3 ─       (@_7 = xi)
└──       

goto #5
4 ─       (@_7 = s)
5 ┄       (s = @_7)
│         (@_4 = Base.iterate(%3, %10))
│   %18 = (@_4 === nothing)::Bool
│   %19 = Base.not_int(%18)::Bool
└──       goto #7 if not %19
6 ─       goto #2
7 ┄       return s



If you don't understand this text, no worries: what you care is that red text means bad type instability, and yellow means type instability that may or may not matter for performance. Whenever a `for` loop is involved there will one yellow line per `for` loop with a `Union` type including `Nothing`. You can ignore this and focus on all other yellow or red colors.

This type instability problem is rather mild here due to the trivial contrived example. It becomes much, much worse if the instability happens in more than one variable, if the instability happens with more than 2 types, or if the types involved in the instability are much more complicated. 

For example, it can become particularly bad once the type instability involves functions.

In [ ]:
function add_funs(x, F)
    F[1](x) + F[2](x)
end

F1 = [sin, cos]
F2 = (sin, cos)

@btime add_funs(1.0, $F1)
@btime add_funs(1.0, $F2);

  89.479 ns (5 allocations: 80 bytes)


  1.200 ns (0 allocations: 0 bytes)


In [ ]:
@code_warntype add_funs(1.0, F1)

MethodInstance for add_funs(::Float64, ::Vector{Function})
  from add_funs(x, F) @ Main c:\Users\datse\OneDrive - University of Exeter\Teaching\Zero2Hero-JuliaWorkshop\1-JuliaIntro.ipynb:1
Arguments
  #self#::Core.Const(add_funs)
  x::Float64
  F::Vector{Function}
Body::Any
1 ─ %1 = Base.getindex(F, 1)::Function
│   %2 = (%1)(x)::Any
│   %3 = Base.getindex(F, 2)::Function
│   %4 = (%3)(x)::Any
│   %5 = (%2 + %4)::Any
└──      return %5



This happens because functions are their own types:

In [ ]:
typeof(cos)

typeof(cos) (singleton type of function cos, subtype of Function)

and wrapping them in a vector creates a vector of their super type:

In [ ]:
typeof([sin, cos])

Vector{Function} (alias for Array{Function, 1})

while the tuple retains the individual types

In [ ]:
typeof((sin, cos))

Tuple{typeof(sin), typeof(cos)}

### Scopes

In general Julia has two scopes: global scope (the one we use here, in this notebook) and local scope. Local scope is introduced by most code blocks, e.g. functions, `for` or `while` loops but *not* from conditional code blocks (`if`). The details of the scopes are mostly relevant for package development and can be found in the [Julia manual](https://docs.julialang.org/en/latest/manual/variables-and-scoping/). 

What is important for us is that by definition, **everything in global scope is type-unstable** and thus not performant. This happens because Julia is not a statically typed language, but a dynamically typed one. Therefore one can do

In [ ]:
x = 5
x = "string"

"string"

which is not possible in e.g. C.

The performance that global scope has in code is truly massive. Here are some global evaluations of some code

In [ ]:
x, y = rand(1000), rand(1000)
a = 0.0
@btime for i in 1:length(x)
    global a += x[i]^2 + y[i]^2
end

  168.500 μs (7980 allocations: 140.33 KiB)


and here are essentially the same operations but done within a function:

In [ ]:
function localf(x, y)
    a = zero(eltype(x))
    for i in 1:length(x)
        a += x[i]^2 + y[i]^2
    end
    return a
end

@btime localf(x, y);

  743.548 ns (1 allocation: 16 bytes)


This is more than a 170x slowdown!!!

### Conclusions so far

1. **Put all performance critical parts of your code inside a function** to avoid global scope
2. **Ensure that your functions are type-stable**, meaning that the types of the variables inside the function do not change. The macro `@code_warntype` can help with that!

## Allocation of mutable containers

Another thing that is important for performance is allocation. What must be understood is that when one writes

In [ ]:
x = rand(2, 2)

2×2 Matrix{Float64}:
 0.635945  0.429758
 0.480919  0.158031

or 

In [ ]:
y = x[1, :]

2-element Vector{Float64}:
 0.6359451105512334
 0.42975762625084135

this *allocates* some part of your memory to store this **mutable** container that `x` or `y` represents. Creating mutable things always allocates memory. In general when you are creating something mutable you always pay two costs:

1. the cost to actually calculate the numbers that go into this thing (here e.g. the cost of calling `rand()`)
2. the cost to allocate some memory to store 1000 numbers of type `Float64`.

In general you should try to avoid allocations, by more clever design of your algorithms and pre-allocating as much as possible, as is instructed by this section of [Julia's performance tips](https://docs.julialang.org/en/latest/manual/performance-tips/#Pre-allocating-outputs-1).

### Example: using `mul!` and `view` to make a non-allocating function

Let's have a look at the following function

In [ ]:
function sum_matrix_vector(A, B)
    out = A*B[:, 1]
    for j in 2:size(B, 2)
        out += A*B[:, j]
    end
    return out
end

n = 50
A = rand(n,n)
B = rand(n,n)
out1 = sum_matrix_vector(A,B)
summary(out1)

"50-element Vector{Float64}"

And let's benchmark its performance

In [ ]:
using BenchmarkTools
@btime sum_matrix_vector($A, $B);

  21.500 μs (149 allocations: 72.17 KiB)


Now we'll make a second version of this function. One that uses in-place multiplication of matrix-vector into a pre-allocated container. And one that uses `view` to view into the matrix `B` without making a copy every time.

In [ ]:
function sum_matrix_vector_nonalloc(A, B)
    # set up
    out = zeros(eltype(B), size(B, 2))
    dummy = copy(out)
    return sum_matrix_vector!(out, dummy, A, B)
end

using LinearAlgebra: mul!
function sum_matrix_vector!(out, dummy, A, B)
    # all computations are in-place
    fill!(out, 0)
    for j in 1:size(B, 2)
        mul!(dummy, A, view(B, :, j))
        out .+= dummy # notice the `.`
    end
    return out
end

sum_matrix_vector! (generic function with 1 method)

In [ ]:
sum_matrix_vector(A, B) == sum_matrix_vector_nonalloc(A, B)

true

In [ ]:
@btime sum_matrix_vector_nonalloc($A, $B);

  14.500 μs (2 allocations: 992 bytes)


As you can see, the improvement in performance is really major! Not only that, but the pre-allocated containers `out, dummy` only need to be made once; different `A, B` do not need re-initialization of `out, dummy` (you'd need to skip the setup part in this case).

## Performance tips summary

1. **Put all performance critical parts of your code inside a function** to avoid global scope
2. **Ensure that your functions are type-stable**, meaning that the types of the variables inside the function do not change. The macro `@code_warntype` can help with that!
3. **Create as little new large mutable entities in your function as possible**, to avoid memory allocations.
4. **Separate your functions into a set-up function and a computing function.** The first initializes all data structures you need, while the second does the actual algorithmic computations. The second function should be **non-allocating**.

# Exercises - performance

## Collatz conjecture
Given a positive integer, create a function that counts the steps it takes to reach `1` following the [Collatz conjecture algorithm](https://en.wikipedia.org/wiki/Collatz_conjecture) (if $n$ is odd do $n=3n+1$ otherwise do $n=n/2$ until you reach 1). Test it with the number 100 to get 25. Ensure that your function is type stable by calling `@code_warntype your_function(100)` and getting no red text.

*Hint: make a type-stable function by using `÷`, (`\div<TAB>`): In Julia `/` is the floating point devision operator and thus `n/m` is always a float number even if `n, m` are integers.*


## Henon map

Create a function that given `u0, N` it creates an orbit of length `N` of the Henon map

$$
\begin{aligned}
x_{n+1} &= 1 - 1.4x^2_n+y_n \\
y_{n+1} &= 0.3x_n
\end{aligned}
$$

This map produces an orbit iteratively, i.e. starting from `u0 = (x0, y0)` one  applies the above rule to get `u1 = (x1, y1)`, and then applies the rule again on `u1` to get `u2`, and so on. The orbit consists of the sequence of states `[u0, u1, u2, ...]`. Use `u0 = (0.0, 0.0)`.

_Hint: The most performant way to represent the states is via Tuples of two floats, not via `Vector`s. To solve, create two functions: one that initializes a length-`N` vector of dummy tuples using `fill`, and one that modifies this pre-initialized vector with the new tuples._
